# 货车GPS数据


## 数据清理流程

In [6]:
import pandas as pd

# 设置输入和输出文件路径
input_file = 'E://20170906.csv'
output_file = './sample1.csv'

# 设置要读取的行数
n = 500000 - 1

# 读取CSV文件的前n行
df = pd.read_csv(input_file, nrows=n, header=None)
# df = pd.read_csv(input_file, header=None)
# n  = len(df)
# df.to_parquet('E://20170906.parquet', index=False)
# 将前n行保存到新CSV文件
# 前 67489999 行已成功保存为 ./sample1.csv .

In [7]:
df = df[(df[2].between(115.41, 117.51)) & (df[3].between(39.44, 41.06))]
len(df)

378012

In [8]:
df = df.drop_duplicates()
len(df)

363793

In [9]:
nunique = df.groupby(df.columns[0])[df.columns[6]].nunique()
nunique.value_counts(2)

6
1    0.974856
2    0.025144
Name: proportion, dtype: float64

In [10]:
df = df[df[1].str.startswith('2017-09-05') | df[1].str.startswith('2017-09-06')]
len(df)

362426

In [11]:
df = df.iloc[:, [0, 1, 2, 3, 4, 7]].rename(columns={
    0: '车号',
    1: '时间',
    2: '经度',
    3: '纬度',
    4: '速度',
    7: '角度'
})
df

,车号,时间,经度,纬度,速度,角度
0,80c3390a-77d1-4d1e-ad9c-6857dbd31fed,2017-09-05 23:58:16,116.636650,39.692375,79,307
1,366abd34-28b8-45c0-bfdd-61cfddaeb8fe,2017-09-05 23:58:15,116.481271,39.863000,49,180
3,d4995584-dbb1-4685-aa55-1da4ce79c8e1,2017-09-05 23:58:05,116.270770,40.211871,0,0
4,a8227ac6-e0f6-42da-a45b-a767d389bc59,2017-09-05 23:58:16,117.367031,40.826500,76,258
5,8cb1a9a6-e627-4ac7-9aab-c69f041922e6,2017-09-05 23:58:15,117.244695,39.969346,0,90
...,...,...,...,...,...,...
499992,858f3712-4d19-4c29-8f2f-4b91e25c96c9,2017-09-06 00:08:57,116.962035,39.744630,0,0
499994,70d2f458-d3b7-456f-a6d4-aa6bb7a0f653,2017-09-06 00:08:57,116.823439,40.394418,55,33
499996,9347972e-45fc-465c-b4fc-8bc8ccd7ebf5,2017-09-06 02:21:37,116.292151,39.771631,45,0
499997,919906a9-5459-441e-83e7-5875899dfa8e,2017-09-06 00:08:59,116.570570,40.130276,0,273


## 目标2

In [12]:
df.to_csv(output_file, index=False)
print(f"预处理数据已成功保存为 {output_file} .")

预处理数据已成功保存为 ./sample1.csv .


In [5]:
import pandas as pd
output_file = './sample1.csv'
df = pd.read_csv(output_file, low_memory=False)

In [6]:
dfv = df[df['速度'] > 2]
len(dfv)

110870

In [7]:
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap

def plot_map(df):
    # 创建更大的地图
    fig, ax = plt.subplots(figsize=(12, 10))  # 设置图的大小为12x10英寸

    # 创建北京市地图
    m = Basemap(projection='merc', llcrnrlat=39.44, urcrnrlat=41.06, llcrnrlon=115.41, urcrnrlon=117.51, ax=ax)
    m.readshapefile(r'map\gadm41_CHN_1', 'prov')
    # 绘制北京市地图，去掉海岸线和国家边界
    m.drawmapboundary(fill_color='aqua')  # 绘制边界并填充颜色
    m.fillcontinents(color='coral', lake_color='blue')  # 填充大陆颜色
    # m.drawrivers(color='blue')  # 绘制河流
    m.drawstates()  # 绘制省级边界

    # 假设df是你的DataFrame，且包含经度和纬度列
    x, y = m(df['经度'].values, df['纬度'].values)

    # 绘制更小的点
    m.scatter(x, y, color='red', marker='.', s=1)  # 设置点的大小为1

    plt.show()

In [8]:
import folium

def map2html(data, name = 'beijing_map.html'):
    # 创建地图对象，设置初始位置和缩放级别
    m = folium.Map(location=[39.9042, 116.4074], zoom_start=10)  # 北京市中心的经纬度
    # folium.Map([40.002694, 116.322373],
    #            tiles= 'https://wprd01.is.autonavi.com/appmaptile?x={x}&y={y}&z={z}&lang=zh_cn&size=1&scl=1&style=8&ltype=11',
    #            attr='高德-街道路网图',
    #            zoom_start=10,
    #           )
    # 遍历DataFrame，为每个数据点创建一个标记
    for index, row in data.iterrows():
        folium.Circle(
            [row['纬度'], row['经度']],
            color='#DC143C',#圈的颜色
            radius = 1,
            fill=True,
            fill_color='#6495ED' #填充颜色
            # ,  # 经度和纬度
            # popup=f"车号: {row['车号']}, 时间: {row['时间']}"  # 弹出窗口显示车号和时间
        ).add_to(m)

    # 保存地图为HTML文件
    m.save('./html/'+name)

# map2html(dfv)
# map2html(df, 'beijing_map_stop.html')
# 北野场桥 = dfv[(dfv['经度'].between(116.425, 116.45)) & (dfv['纬度'].between(39.715, 39.73))]
# map2html(北野场桥, '北野场桥.html')

### Haversine公式

In [9]:
import math
# 计算两点之间的球面距离
def haversine_distance(lat1, lon1, lat2, lon2):
    # 将十进制度数转换为弧度
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])

    # haversine公式
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a))
    r = 6371393  # 地球平均半径，单位为米
    return c * r   # 结果单位为米

### 寻找1km对应的经纬度差

In [10]:
north = [39.9152097,116.3912757]
east = [39.906217,116.40299870]
center = [39.906217, 116.3912757]
print(haversine_distance(center[0],center[1],north[0],north[1])-1000)
print(haversine_distance(center[0],center[1],east[0],east[1])-1000)
deltaLat = north[0] - center[0]
deltaLon = east[1] - center[1]
print(deltaLat,deltaLon)

0.004299060397329413
-2.971175115362712e-05
0.008992700000000298 0.01172300000000348


In [11]:
# df = df.rename(columns={'纬度': "latitude", '经度': "longitude"})
dfv = df[df['速度'] > 2]

In [23]:
import numpy as np
import pandas as pd

# 定义北京市范围
BEIJING_BOUNDARY = {
    "lat_min": 39.44,
    "lat_max": 41.06,
    "lon_min": 115.41,
    "lon_max": 117.51,
}

# 定义近似正方形单位（1平方公里）
GRID_SIZE_LAT = 0.0089927  # 纬度差
GRID_SIZE_LON = 0.011723  # 经度差


def create_grid(boundary, grid_size_lat, grid_size_lon):
    """创建北京市划分网格."""
    lat_range = np.arange(boundary["lat_min"], boundary["lat_max"], grid_size_lat)
    lon_range = np.arange(boundary["lon_min"], boundary["lon_max"], grid_size_lon)
    return lat_range, lon_range


def assign_points_to_grid(lat_range, lon_range, window = 20):
    """将点分配到网格."""
    global df
    df['时间'] = pd.to_datetime(df['时间'])
    grid_counts = np.zeros((len(lat_range) - 2, len(lon_range) - 2))

    df['lat_idx'] = np.searchsorted(lat_range, df["纬度"], side="right") - 1
    df['lon_idx'] = np.searchsorted(lon_range, df["经度"], side="right") - 1
    df['valid'] = False
    df['outTime'] = False

    grouped = df.groupby('车号')
    for idxs, (car_id, group) in enumerate(grouped):
        group = group.sort_values('时间') 
        pastList = []
        futureList = [(a, b) for a, b in group.iloc[:min(window, len(group))][['lat_idx', 'lon_idx']].values]
        for num, (rowidx, row) in enumerate(group.iterrows()):
            lat_idx = row['lat_idx']
            lon_idx = row['lon_idx']            
            # 检查当前数据的lat_idx和lon_idx是否与未来二十条数据均不同，不同说明车辆已经离开当前网格
            futureList.pop(0)
            if num+window < len(group):
                futureList.append(tuple(group.iloc[num+window][['lat_idx', 'lon_idx']].values))
            if (lat_idx, lon_idx) not in futureList:
                df.loc[rowidx, 'outTime'] = True            
            # 检查当前数据的lat_idx和lon_idx是否与过去二十条数据均不同
            pastList.append((lat_idx, lon_idx))
            if (lat_idx, lon_idx) not in pastList[:-1]:
                df.loc[rowidx, 'valid'] = True
                if 0 <= lat_idx < grid_counts.shape[0] and 0 <= lon_idx < grid_counts.shape[1]:
                    grid_counts[lat_idx, lon_idx] += 1
                if 0 <= lat_idx - 1 < grid_counts.shape[0] and 0 <= lon_idx - 1 < grid_counts.shape[1]:
                    grid_counts[lat_idx - 1, lon_idx - 1] += 1
                if 0 <= lat_idx < grid_counts.shape[0] and 0 <= lon_idx - 1 < grid_counts.shape[1]:
                    grid_counts[lat_idx, lon_idx - 1] += 1
                if 0 <= lat_idx - 1 < grid_counts.shape[0] and 0 <= lon_idx < grid_counts.shape[1]:
                    grid_counts[lat_idx - 1, lon_idx] += 1
            if num >= window:
                pastList.pop(0)
    return grid_counts


In [24]:
lat_range, lon_range = create_grid(BEIJING_BOUNDARY, GRID_SIZE_LAT, GRID_SIZE_LON)
grid_counts = assign_points_to_grid(lat_range, lon_range)
# max_count, max_grid = find_max_grid(grid_counts, lat_range, lon_range)

In [31]:
# 将二维数组展平为一维数组
flat_grid = grid_counts.flatten()

# 获取前100个最大值及其索引
top_100_indices = np.argpartition(flat_grid, -1000)[-1000:]
top_100_values = flat_grid[top_100_indices]

# 获取对应的二维索引
top_100_coords = np.unravel_index(top_100_indices, grid_counts.shape)

# 创建 DataFrame 以便更好地查看数据
df_top_100 = pd.DataFrame({
    'Value': top_100_values,
    'Row': top_100_coords[0],
    'Column': top_100_coords[1]
})

# 按值排序
df_top_100 = df_top_100.sort_values(by='Value', ascending=False).reset_index(drop=True)

print(df_top_100)

threshold = df_top_100['Value'][19]
threshold

     Value  Row  Column
0    862.0   40      78
1    623.0   40      77
2    617.0   41      78
3    410.0   39      78
4    401.0   43      99
..     ...  ...     ...
995   83.0   36     101
996   83.0   66     107
997   83.0   98     118
998   83.0   90      58
999   83.0   23      84

[1000 rows x 3 columns]


298.0

In [32]:
dense_regions = df_top_100[df_top_100['Value'] >= threshold][['Row', 'Column']].values.tolist()
dense_regions

[[40, 78],
 [40, 77],
 [41, 78],
 [39, 78],
 [43, 99],
 [42, 99],
 [28, 78],
 [34, 98],
 [41, 77],
 [40, 79],
 [44, 95],
 [39, 77],
 [34, 97],
 [41, 79],
 [46, 95],
 [43, 100],
 [28, 77],
 [34, 101],
 [46, 96],
 [27, 78],
 [47, 95],
 [34, 100]]

In [33]:
def findSubDense(regions:list, threshold = np.iinfo(np.int32).max):
    """
    寻找regions列表中每个元素所在区域(2km * 2km)的最密子区域(1km * 1km)
    返回最密子区域的列表和点最少的子区域所含的点
    """
    minThreshold = threshold
    subDense = [] # 存储最密子区域的经纬度范围
    for latidx, lonidx in regions:
        maxCount = 0        
        bound = (lat_range[latidx], lat_range[latidx+ 2], lon_range[lonidx], lon_range[lonidx + 2])
        dfRegion = df[(df['经度'].between(bound[2], bound[3])) & (df['纬度'].between(bound[0], bound[1]))]
        # return dfRegion, 0
        for ilat in range(11):
            for ilon in range(11):
                lon = bound[2] + GRID_SIZE_LON/10 * ilon
                lat = bound[0] + GRID_SIZE_LAT/10 * ilat
                count = len(dfRegion[
                    (dfRegion['经度'].between(lon, lon + GRID_SIZE_LON)) &
                    (dfRegion['纬度'].between(lat, lat + GRID_SIZE_LAT)) &
                    (dfRegion['valid'] == True)
                ])
                if maxCount < count:
                    maxCount = count
                    latRegion = lat
                    lonRegion = lon
        subDense.append([latRegion, lonRegion, maxCount])
        minThreshold = min(minThreshold, maxCount)
        # print(f"最密集区域的数据点数量: {maxCount}")
        # print(f"最密集区域的纬度范围: ({latRegion}, {latRegion + GRID_SIZE_LAT})")
        # print(f"最密集区域的经度范围: ({lonRegion}, {lonRegion + GRID_SIZE_LON})")
    print(f"最小阈值: {minThreshold}")
    subDense = pd.DataFrame(subDense, columns=["纬度", "经度", "计数"])
    return subDense, minThreshold

In [34]:
subDense, minThreshold = findSubDense(dense_regions, threshold)

最小阈值: 118


In [35]:
reference_regions = df_top_100[df_top_100['Value'] >= minThreshold][['Row', 'Column']].values.tolist()
reference_regions

[[40, 78],
 [40, 77],
 [41, 78],
 [39, 78],
 [43, 99],
 [42, 99],
 [28, 78],
 [34, 98],
 [41, 77],
 [40, 79],
 [44, 95],
 [39, 77],
 [34, 97],
 [41, 79],
 [46, 95],
 [43, 100],
 [28, 77],
 [34, 101],
 [46, 96],
 [27, 78],
 [47, 95],
 [34, 100],
 [41, 92],
 [35, 97],
 [27, 77],
 [29, 78],
 [47, 96],
 [81, 69],
 [43, 95],
 [45, 95],
 [43, 94],
 [42, 93],
 [81, 70],
 [35, 101],
 [49, 95],
 [42, 100],
 [45, 96],
 [33, 98],
 [44, 96],
 [76, 98],
 [50, 95],
 [35, 100],
 [54, 95],
 [75, 72],
 [64, 79],
 [61, 92],
 [49, 96],
 [82, 69],
 [35, 98],
 [42, 92],
 [54, 96],
 [33, 93],
 [50, 96],
 [54, 97],
 [55, 98],
 [80, 70],
 [63, 79],
 [35, 84],
 [69, 98],
 [54, 98],
 [41, 115],
 [76, 97],
 [55, 95],
 [75, 71],
 [74, 72],
 [32, 93],
 [75, 98],
 [33, 97],
 [44, 108],
 [42, 115],
 [75, 97],
 [84, 68],
 [51, 95],
 [86, 67],
 [92, 57],
 [92, 56],
 [48, 95],
 [48, 96],
 [45, 68],
 [52, 96],
 [41, 93],
 [41, 91],
 [44, 99],
 [101, 59],
 [80, 69],
 [30, 84],
 [36, 85],
 [93, 55],
 [46, 109],
 [42, 94],

In [36]:
subDense, minThreshold = findSubDense(reference_regions)
subDense.sort_values(by = '计数', ascending = False, inplace=True)
subDense

最小阈值: 40


,纬度,经度,计数
0,39.806003,116.332600,429
2,39.808701,116.330256,392
1,39.802406,116.324394,323
8,39.808701,116.324394,291
3,39.799708,116.331428,279
...,...,...,...
510,40.069489,116.451002,50
463,39.858161,116.564716,49
387,39.774528,116.535408,48
319,39.871650,116.570577,45


<!-- 参考区域数量: 1
阈值：929452.1989
纬度范围: (39.799708, 39.817693), 经度范围: (116.324394, 116.347840), 点个数：930855

最密集区域的数据点数量: 3650
最密集区域的纬度范围: (39.80600289000001, 39.81499559000001)
最密集区域的经度范围: (116.33260010000026, 116.34432310000027)

备选区域数量: 3
阈值：452968.0
纬度范围: (39.799708, 39.8176934), 经度范围: (116.312671, 116.336117), 点个数：729496
纬度范围: (39.799708, 39.8176934), 经度范围: (116.324394, 116.347840), 点个数：930855
纬度范围: (39.808701, 39.826686), 经度范围: (116.324394, 116.347840), 点个数：666237

最密集区域的数据点数量: 414776
最密集区域的纬度范围: (39.802406, 39.811399)
最密集区域的经度范围: (116.324394, 116.336117)
最密集区域的数据点数量: 452968
最密集区域的纬度范围: (39.80600289000001, 39.81499559000001)
最密集区域的经度范围: (116.33260010000026, 116.34432310000027)
最密集区域的数据点数量: 429662
最密集区域的纬度范围: (39.808700, 39.817693)
最密集区域的经度范围: (116.330256, 116.341979) -->

In [37]:
Top20 = subDense[:20]
print(f"阈值：{Top20['计数'].min()}")
print(Top20)

阈值：170
           纬度          经度   计数
0   39.806003  116.332600  429
2   39.808701  116.330256  392
1   39.802406  116.324394  323
8   39.808701  116.324394  291
3   39.799708  116.331428  279
11  39.799708  116.324394  265
9   39.806003  116.336117  253
5   39.824888  116.575266  251
4   39.826686  116.575266  231
13  39.808701  116.336117  216
35  39.823089  116.582300  204
10  39.841974  116.534236  197
53  39.931901  116.555337  191
59  39.931901  116.558854  183
38  39.841974  116.535408  175
17  39.750248  116.595195  173
21  39.750248  116.594023  171
6   39.697191  116.324394  170
15  39.826686  116.582300  170
16  39.697191  116.324394  170


In [39]:
def odAnalysis(regions, timeThreshold = 600, lat_range = lat_range, lon_range = lon_range):
    """
    对regions中的每个区域进行OD分析
    """
    allOrigin = []
    allIntime = []
    allDest = []
    allOuttime = []
    for lat, lon, _ in regions:
        origin = []
        dest = []
        inTime = []
        outTime = []
        bound = (lat, lat + GRID_SIZE_LAT, lon, lon + GRID_SIZE_LON)
        dfRegion = df[(df['经度'].between(bound[2], bound[3])) & (df['纬度'].between(bound[0], bound[1]))]
        grouped = dfRegion.groupby('车号')
        for car_id, group in grouped:
            group = group.sort_values(by='时间')
            for idx, row in group.iterrows():
                if row['outTime']:
                    outTime.append(row['时间'])
                if not row['valid'] :
                    continue                
                inTime.append(row['时间'])
                def findStop(stop, arrive, types):
                    """找到最近的连续600秒停车的记录"""
                    for _, rowStop in stop.iterrows():
                        # 静止
                        if rowStop['速度'] < 2:
                            # 静止时间超过timeThreshold
                            # 且车辆位置发生变化
                            if (arrive - rowStop['时间']).total_seconds() > timeThreshold and \
                                (rowStop['lon_idx'] != row['lon_idx'] or rowStop['lat_idx'] != row['lat_idx']): 
                                # print(f"车号: {rowStop['车号']}{types}信息：\n \
                                #     由 {rowStop['时间']}, 纬度: {rowStop['纬度']}, 经度: {rowStop['经度']}\n \
                                #     到 {row['时间']}, 纬度: {row['纬度']}, 经度: {row['经度']}")
                                lat_idx = np.searchsorted(lat_range, rowStop['纬度'], side="right") - 1
                                lon_idx = np.searchsorted(lon_range, rowStop['经度'], side="right") - 1
                                return [rowStop['车号'], rowStop['时间'], lat_idx, lon_idx]
                        else:
                            arrive = rowStop['时间']
                    return None
                before_stop = group.loc[:idx]
                arrive = row['时间']
                stop = findStop(before_stop[::-1], arrive, "进入")
                if stop is not None:
                    origin.append(stop)
                
                # 找到之后最近的连续600秒停车的记录
                after_stop = group.loc[idx:]
                stop = findStop(after_stop, arrive, "离开")
                if stop is not None:
                    dest.append(stop)
    
        print(f"区域({lat}, {lon})的OD分析结果：")
        print(f"进入区域的车辆数：{len(origin)}")
        print(f"离开区域的车辆数：{len(dest)}")
        print(f"进入区域的车辆信息：{origin}")
        print(f"离开区域的车辆信息：{dest}")
        print(f"进入区域的时间：{inTime}")
        print(f"离开区域的时间：{outTime}")
        allOrigin.append(origin)
        allDest.append(dest)
        allIntime.append(inTime)
        allOuttime.append(outTime)
    return allOrigin, allDest, allIntime, allOuttime

In [40]:
allOrigin, allDest, allIntime, allOuttime = odAnalysis(Top20[['纬度', '经度', '计数']].values, 60)
# 每一个都是二维列表，第一维是区域，第二维是具体数据
# allOrigin和allDest的每个元素是一个列表，列表中的每个元素是一个列表，包含车号、时间、纬度、经度
# allIntime和allOuttime的每个元素是一个列表，列表中的每个元素是一个时间戳

区域(39.80600289000001, 116.33260010000026)的OD分析结果：
进入区域的车辆数：6
离开区域的车辆数：0
进入区域的车辆信息：[['15b09405-9efe-46a1-9ac6-4836b7a80a86', Timestamp('2017-09-06 00:00:19'), 41, 79], ['44ab64f7-98aa-4ad9-805b-b4d628debe59', Timestamp('2017-09-05 23:59:41'), 41, 79], ['56928225-9bad-4ddd-bdcf-c8697f966c2c', Timestamp('2017-09-05 23:59:43'), 41, 79], ['e393f986-1c42-4538-bd29-f4d390cd6e30', Timestamp('2017-09-06 00:04:22'), 41, 78], ['e393f986-1c42-4538-bd29-f4d390cd6e30', Timestamp('2017-09-06 00:04:22'), 41, 78], ['eb4f64d2-93e4-4936-83c5-38e57a9948bd', Timestamp('2017-09-06 00:03:48'), 40, 79]]
离开区域的车辆信息：[]
区域(39.80870070000001, 116.33025550000026)的OD分析结果：
进入区域的车辆数：2
离开区域的车辆数：0
进入区域的车辆信息：[['e393f986-1c42-4538-bd29-f4d390cd6e30', Timestamp('2017-09-06 00:04:22'), 41, 78], ['fd69691b-55c4-4b3f-9529-1d44c61bbd63', Timestamp('2017-09-06 00:01:19'), 41, 79]]
离开区域的车辆信息：[]
区域(39.80240581000001, 116.32439400000027)的OD分析结果：
进入区域的车辆数：2
离开区域的车辆数：0
进入区域的车辆信息：[['24515b3c-9cb4-4c2e-900c-341a1fc6930a', Timestamp('2

In [75]:
import folium

def map2html2(data, name = 'map.html'):
    # 创建地图对象，设置初始位置和缩放级别
    m = folium.Map([39.908722, 116.397499], zoom_start=12)

    # 添加20个密集区域的标记
    i = 1
    for lat, lon in data[['纬度', '经度']].values:
        folium.Polygon([
            [lat, lon],
            [lat, lon + GRID_SIZE_LON],
            [lat + GRID_SIZE_LAT, lon + GRID_SIZE_LON],
            [lat + GRID_SIZE_LAT, lon],
            [lat, lon],
        ],
        color='#FF0000',  # 多边形边界颜色
        fill=False
    ).add_to(m)
        # 在多边形左上角添加标记
        folium.Marker(
            [lat + GRID_SIZE_LAT * 9.5 / 10, lon + GRID_SIZE_LON * 0.5 / 10],
            icon=folium.DivIcon(html=f'''
                <div class ="my-label" style="font-size: 12pt; color: red; white-space: nowrap;">
                    第{i+1}聚集地
                </div>
            ''')
        ).add_to(m)
        i += 1
    # 调整代码以适应不同的分辨率
    m.get_root().html.add_child(folium.Element('''
        <script>
        function updateLabelSize() {
            var zoom = map.getZoom();
            var labels = document.querySelectorAll('.my-label');
            labels.forEach(function(label) {
                label.style.fontSize = (zoom * 1.5) + 'pt';
            });
        }
        map.on('zoomend', updateLabelSize);
        updateLabelSize();
        </script>
    '''))
    # 保存地图到HTML文件
    m.save('./html/' + name)
    

map2html2(Top20, 'Top20.html')
# map2html2(dfResult)
# 北野场桥 = dfv[(dfv['经度'].between(116.425, 116.45)) & (dfv['纬度'].between(39.715, 39.73))]
# map2html(北野场桥, '北野场桥.html')

In [56]:
import matplotlib.pyplot as plt
from collections import Counter

plt.rcParams['font.sans-serif'] = ['SimHei']  # 使用黑体
plt.rcParams['axes.unicode_minus'] = False  # 解决负号显示问题

# 分析每个区域的进入时间、离开时间分布
def InOutAnalysis(intime, outtime, name):
    plt.figure(figsize=(10, 5))
    plt.hist(intime, bins=24, alpha=0.5, label='进入时间')
    plt.hist(outtime, bins=24, alpha=0.5, label='离开时间')
    plt.title(f'区域 {i+1} 的进入时间和离开时间分布')
    plt.xlabel('时间')
    plt.ylabel('频率')
    plt.legend()
    plt.savefig(f'img/时序分布/' + name + '时间分布.png')
    plt.close()

for i in range(len(Top20)):
    InOutAnalysis(allIntime[i], allOuttime[i], f'区域{i+1}')


In [57]:
# 分析所有区域的进入时间、离开时间分布
all_intime = [time for sublist in allIntime for time in sublist]
all_outtime = [time for sublist in allOuttime for time in sublist]
InOutAnalysis(all_intime, all_outtime, '所有区域')

In [ ]:

def ODAnalysis(origin, dest, name):
# 统计出发地和目的地的频率
    origin_counter = Counter((lat, lon) for _, _, lat, lon in origin)
    dest_counter = Counter((lat, lon) for _, _, lat, lon in dest)

# 取前20名
    top_20_origins = origin_counter.most_common(20)
    top_20_dests = dest_counter.most_common(20)

# 分别获取纬度、经度和计数
    if top_20_origins:
        origin_lats, origin_lons, origin_counts = zip(*[(lat, lon, count) for (lat, lon), count in top_20_origins])
        origin_labels = [f'({lat}, {lon})' for lat, lon in zip(origin_lats, origin_lons)]
    else:
        origin_counts = []
        origin_labels = []

    if top_20_dests:
        dest_lats, dest_lons, dest_counts = zip(*[(lat, lon, count) for (lat, lon), count in top_20_dests])
        dest_labels = [f'({lat}, {lon})' for lat, lon in zip(dest_lats, dest_lons)]
    else:
        dest_counts = []
        dest_labels = []

# 绘制图表
    plt.figure(figsize=(10, 5))
    if origin_counts:
        plt.bar(range(len(origin_counts)), origin_counts, alpha=0.5, label='出发地')
    if dest_counts:
        plt.bar(range(len(dest_counts)), dest_counts, alpha=0.5, label='目的地')

# 设置横轴标签
    if origin_labels or dest_labels:
        plt.xticks(range(max(len(origin_labels), len(dest_labels))), origin_labels + dest_labels, rotation=0)

    plt.title(f'区域 {i+1} 的出发地和目的地分布')
    plt.xlabel('区域坐标')
    plt.ylabel('频率')
    plt.legend()
    plt.tight_layout()  # 调整布局以防止标签重叠
    plt.savefig("img/OD分布/" + name + "出发地和目的地分布.png")
    plt.close()

for i in range(len(Top20)):
    ODAnalysis(allOrigin[i], allDest[i], "区域" + str(i+1))
    # break

In [ ]:
all_origins = [item for sublist in allOrigin for item in sublist]
all_dests = [item for sublist in allDest for item in sublist]
ODAnalysis(all_origins, all_dests, "Top20")